In [1]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt
from scipy.signal import hilbert
from antropy import sample_entropy
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from scipy.stats import mode


In [10]:
bands = {
    "delta": (0.5, 4),
    "theta": (4, 8),
    "alpha": (8, 13),
    "beta": (13, 30),
    "gamma": (30, 45)
}

def bandpower(sig, fs, band):  #filtrira signale po frekvencijama
    low, high = band
    b, a = butter(4, [low / (fs / 2), high / (fs / 2)], btype='band')
    filtered = filtfilt(b, a, sig)
    power = np.sum(filtered ** 2) / len(filtered)
    return power 

In [9]:
def compute_esis(epoch, fs):
    
    energy = np.square(epoch)
    slope = np.abs(np.diff(epoch, prepend=epoch[0]))
    v = fs * slope
    esis = np.sum(energy * v)

    return esis

In [22]:
def compute_features(epoch, fs = 100):
    rms = np.sqrt(np.mean(np.square(epoch))) #ROOT MEAN SQUARE
    mse = np.mean((epoch - np.mean(epoch)) ** 2) #MEAN SQUARED ERROR
    mmd = np.mean(np.abs(np.diff(epoch))) #MINIMAL MAXIMAL DISTANCE
    zcr = ((epoch[:-1] * epoch[1:]) < 0).sum() #ZERO CROSSING RATE
    se = sample_entropy(epoch) #SAMPLE ENTROPY
    esis = compute_esis(epoch, fs)

    talasi = {ime: bandpower(epoch, fs, opseg) for ime, opseg in bands.items()}

    red = {
        "RMS": rms,
        "MSE": mse,
        "MMD": mmd,
        "ZCR": zcr,
        "Sample Entropy": se,
        "Esis": esis,
        "Delta": talasi["delta"],
        "Theta": talasi["theta"],
        "Alpha": talasi["alpha"],
        "Beta": talasi["beta"],
        "Gamma": talasi["gamma"]
    }

    return [rms, mse, mmd, zcr, se, esis, talasi["alpha"], talasi["beta"], talasi["gamma"], talasi["delta"], talasi["theta"]]

In [39]:
folder = "C:/Analiza Faza Sna/"
files = [f for f in os.listdir(folder) if f.endswith(".csv")]

all_subjects = []

for file in files:
    df = pd.read_csv(os.path.join(folder, file))
    df = df[df["Sleep_Stage"] != "P"].copy()

    signal = df["F4-M1"].values
    labels = df["Sleep_Stage"].values
    fs = 100
    samples_per_epoch = fs * 30
    num_epochs = len(signal) // samples_per_epoch

    signal_epochs = np.array_split(signal[:num_epochs * samples_per_epoch], num_epochs)
    label_epochs = np.array_split(labels[:num_epochs * samples_per_epoch], num_epochs)

    normalized_epochs = []

    for epoch in signal_epochs:
        mean = np.mean(epoch)
        std = np.std(epoch)
        if std != 0:
            norm_epoch = (epoch - mean) / std
        else:
            norm_epoch = epoch - mean
        normalized_epochs.append(norm_epoch)

    epoch_labels = [pd.Series(epoch).mode()[0] for epoch in label_epochs]
    features = [compute_features(epoch) for epoch in normalized_epochs]
    all_subjects.append((np.array(features), np.array(epoch_labels)))


In [40]:
print(len(all_subjects))

14


In [44]:
le = LabelEncoder()

reports = []

for i in range(len(all_subjects)):
    X_test, y_test = all_subjects[i]
    X_train = np.vstack([all_subjects[j][0] for j in range(len(all_subjects)) if j != i])
    y_train = np.concatenate([all_subjects[j][1] for j in range(len(all_subjects)) if j != i])

    y_train_enc = le.fit_transform(y_train)
    y_test_enc = le.transform(y_test)

    clf = RandomForestClassifier(n_estimators=100, random_state=42)
    clf.fit(X_train, y_train_enc)

    y_pred = clf.predict(X_test)

    print(f"\nSubject {i + 1} - {files[i]}")
    unique_labels = np.unique(np.concatenate([y_test_enc, y_pred]))
    print(classification_report(
        y_test_enc,
        y_pred,
        labels=unique_labels,
        target_names=le.inverse_transform(unique_labels)
    ))





Subject 1 - S002_PSG_df_updated.csv
              precision    recall  f1-score   support

          N1       0.00      0.00      0.00        64
          N2       0.41      0.63      0.50       334
          N3       0.00      0.00      0.00         0
           R       0.00      0.00      0.00        80
           W       0.68      0.28      0.39       264

    accuracy                           0.38       742
   macro avg       0.22      0.18      0.18       742
weighted avg       0.43      0.38      0.36       742



c:\Users\andre\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\andre\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\andre\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metri


Subject 2 - S003_PSG_df_updated.csv
              precision    recall  f1-score   support

          N1       0.00      0.00      0.00        32
          N2       0.62      0.54      0.57       428
          N3       1.00      0.07      0.14       136
           R       0.52      0.13      0.20       119
           W       0.24      0.86      0.38       113

    accuracy                           0.42       828
   macro avg       0.48      0.32      0.26       828
weighted avg       0.59      0.42      0.40       828


Subject 3 - S004_PSG_df_updated.csv
              precision    recall  f1-score   support

          N1       1.00      0.01      0.02       163
          N2       0.64      0.84      0.72       423
          N3       0.00      0.00      0.00         1
           R       0.00      0.00      0.00         0
           W       0.53      0.26      0.35       248

    accuracy                           0.51       835
   macro avg       0.43      0.22      0.22       835
wei

c:\Users\andre\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\andre\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\andre\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metri


Subject 4 - S005_PSG_df_updated.csv
              precision    recall  f1-score   support

          N1       0.18      0.08      0.11        39
          N2       0.71      0.81      0.76       479
          N3       0.02      0.40      0.04         5
           R       0.52      0.12      0.20       116
           W       0.75      0.42      0.54        93

    accuracy                           0.61       732
   macro avg       0.44      0.37      0.33       732
weighted avg       0.65      0.61      0.60       732


Subject 5 - S006_PSG_df_updated.csv
              precision    recall  f1-score   support

          N1       0.35      0.15      0.21       122
          N2       0.54      0.75      0.63       290
          N3       0.70      0.18      0.29       178
           R       0.00      0.00      0.00         0
           W       0.61      0.38      0.47       252

    accuracy                           0.43       842
   macro avg       0.44      0.29      0.32       842
wei

c:\Users\andre\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\andre\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\andre\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metri


Subject 6 - S007_PSG_df_updated.csv
              precision    recall  f1-score   support

          N1       0.27      0.12      0.16        26
          N2       0.77      0.94      0.84       299
          N3       0.64      0.35      0.45        26
           R       0.23      0.46      0.30        41
           W       0.99      0.79      0.88       428

    accuracy                           0.80       820
   macro avg       0.58      0.53      0.53       820
weighted avg       0.84      0.80      0.80       820


Subject 7 - S008_PSG_df_updated.csv
              precision    recall  f1-score   support

          N1       0.40      0.38      0.39        42
          N2       0.78      0.92      0.84       421
          N3       0.78      0.20      0.32        89
           R       0.83      0.60      0.69       137
           W       0.57      0.88      0.69        58

    accuracy                           0.74       747
   macro avg       0.67      0.60      0.59       747
wei

c:\Users\andre\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\andre\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\andre\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metri


Subject 12 - S013_PSG_df_updated.csv
              precision    recall  f1-score   support

          N1       0.15      0.11      0.12        19
          N2       0.77      0.98      0.86       495
          N3       0.17      0.01      0.02        82
           R       0.94      0.64      0.76       152
           W       0.68      0.68      0.68        37

    accuracy                           0.78       785
   macro avg       0.54      0.48      0.49       785
weighted avg       0.72      0.78      0.73       785


Subject 13 - S014_PSG_df_updated.csv
              precision    recall  f1-score   support

          N1       0.15      0.16      0.15        31
          N2       0.66      0.95      0.78       431
          N3       1.00      0.02      0.04       106
           R       0.21      0.36      0.27        33
           W       0.88      0.36      0.51       194

    accuracy                           0.63       795
   macro avg       0.58      0.37      0.35       795
w